In [ ]:
!pip install sovai[full]

## Clinical Trials

In [2]:
import sovai as sov
import pandas as pd

sov.token_auth(token="visit https://sov.ai/profile for your token")

Prediction Data

In [3]:
df_apps = sov.data("clinical/predict", tickers=["PFE"], start_date = "2024-01-01", end_date= "2025-02-01",  purge_cache=True)

Loading partitions: 100%|██████████| 1/1 [00:00<00:00,  1.83partition/s]


Trial Descriptions

In [ ]:
df_trials = sov.data("clinical/trials", tickers=["PFE","LLY"], start_date="2022", purge_cache=True); df_trials.tail()

Visual Analysis

In [25]:
df = sov.data("clinical/predict", tickers=["PFE","LLY"], start_date="2022", purge_cache=True)

Loading partitions: 100%|██████████| 2/2 [00:00<00:00,  4.44partition/s]


In [41]:
import pandas as pd
from statsmodels.nonparametric.smoothers_lowess import lowess
import plotly.express as px
import plotly.graph_objects as go

# ------------- CONFIG -------------
metric   = 'success_prediction'
tickers  = ['LLY', 'PFE']          # order matters: LLY first
colors   = {'LLY': '#1f77b4', 'PFE': '#ff7f0e'}

# ------------- DATA & LOWESS -------------
df = df_apps.reset_index().sort_values(['ticker', 'date'])

def add_loess(g, y=metric, frac=0.2):
    x = (g['date'] - pd.Timestamp('1970-01-01')).dt.days
    g['loess'] = lowess(g[y], x, frac=frac)[:, 1]
    return g

df = df.groupby('ticker', group_keys=False).apply(add_loess)

# ------------- FIGURE 1: shared y‑axis -----------------
fig_shared = px.scatter(
    df, x='date', y=metric, color='ticker',
    template='plotly_dark', opacity=0.35,
    color_discrete_map=colors,
    hover_data={'ticker': False, metric: ':.3f'}
)

# overlay LOWESS lines
for t in tickers:
    sub = df[df['ticker'] == t]
    fig_shared.add_scatter(
        x=sub['date'], y=sub['loess'],
        mode='lines', name=f'{t} trend',
        line=dict(color=colors[t], width=3)
    )

fig_shared.update_layout(
    title=f'{metric.replace("_", " ").title()} – Shared Y‑axis',
    xaxis_title='Date', yaxis_title=metric.replace('_', ' ').title(),
    legend_orientation='h', legend_y=1.05, legend_x=1, legend_xanchor='right',
    margin=dict(t=80, b=40, l=60, r=40), height=450
).update_xaxes(rangeslider_visible=False)

fig_shared.show()

# ------------- FIGURE 2: twin y‑axes -------------------
fig_twin = go.Figure()

for i, t in enumerate(tickers):
    g  = df[df['ticker'] == t]
    ax = 'y' if i == 0 else 'y2'

    # raw dots
    fig_twin.add_scatter(
        x=g['date'], y=g[metric], mode='markers',
        marker=dict(color=colors[t], size=6, opacity=0.4),
        name=f'{t} raw', hoverinfo='none', yaxis=ax
    )
    # trend
    fig_twin.add_scatter(
        x=g['date'], y=g['loess'], mode='lines',
        line=dict(color=colors[t], width=3), name=f'{t} trend', yaxis=ax,
        hovertemplate=f"<b>{t} trend</b><br>Date: %{{x|%b %d, %Y}}<br>{metric}: %{{y:.3f}}<extra></extra>"
    )

    # end‑value annotation
    end_val = g['loess'].iloc[-1]
    fig_twin.add_annotation(
        x=g['date'].iloc[-1], y=end_val, yref=ax,
        text=f"{t}: {end_val:.2f}", showarrow=False,
        font=dict(color=colors[t]), bgcolor='rgba(0,0,0,0.6)', xanchor='left'
    )

# compute tight ranges
rng = {
    t: (df.loc[df['ticker'] == t, 'loess'].min()*0.99,
        df.loc[df['ticker'] == t, 'loess'].max()*1.01)
    for t in tickers
}

fig_twin.update_layout(
    template='plotly_dark',
    title=f'{metric.replace("_", " ").title()} – Twin Y‑axes',
    xaxis=dict(title='Date',
               showgrid=True, gridcolor='rgba(255,255,255,0.1)'),

    # left axis = LLY
    yaxis=dict(
        title=dict(text=f'LLY {metric}', font=dict(color=colors["LLY"])),
        tickfont=dict(color=colors["LLY"]),
        range=rng['LLY'],
        showgrid=False
    ),

    # right axis = PFE
    yaxis2=dict(
        title=dict(text=f'PFE {metric}', font=dict(color=colors["PFE"])),
        tickfont=dict(color=colors["PFE"]),
        overlaying='y', side='right',
        range=rng['PFE'],
        showgrid=False
    ),

    legend=dict(
        orientation='h', y=1.05, x=1, xanchor='right',
        bgcolor='rgba(30,30,30,0.6)', borderwidth=1
    ),

    margin=dict(t=80, b=40, l=60, r=60),
    height=500
)

fig_twin.update_xaxes(rangeslider_visible=False)
fig_twin.show()



In [12]:
df_apps = sov.data("clinical/trials", start_date="2025-01-20", verbose=False)

In [15]:
df_apps.sample(5)

,,source,subsidiary,sponsor,trial_id,official_title,sponsor_study_id,brief_title,lead_sponsor,lead_sponsor_name,sponsor_type,lead_sponsor_type,phase_category,study_type,enrollment_count,number_of_locations,healthy_volunteers,enrollment_type,study_size_category,condition_keywords,primary_condition,intervention_type,primary_intervention,intervention_name,intervention_arm_group_labels,intervention_description,has_data_monitoring_committee,responsible_party_investigator_affiliation,responsible_party_investigator_title,responsible_party_investigator_name,first_posted_date,last_update_posted_date,start_date,study_completion_date,primary_completion_date,study_locations_city,study_locations_country,study_locations_geopoint,study_locations_facility,study_locations_zip,study_locations_state,standard_age_groups,sex,minimum_age,maximum_age,overall_status,status_category,status_verified_date,primary_outcomes_measures,primary_outcomes_timeframes,primary_outcomes_descriptions,secondary_outcomes_measures,secondary_outcomes_timeframes,secondary_outcomes_descriptions,has_results,conditions,brief_summary,detailed_description,masking,allocation,intervention_model,primary_purpose,has_expanded_access,study_duration_days,trial_duration,references_type,references_citation,references_pmid,collaborators_name,collaborators_class,ipd_sharing,success_prediction,economic_effect,duration_prediction,success_composite,class
ticker,date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GOV,2025-01-27,government,Istituti Clinici Scientifici Maugeri SpA,Istituti Clinici Scientifici Maugeri SpA,NCT06823193,How Self-Processing Affects Mental Time Travel...,ICS Maugeri CE 2717,The Impact Of Self-Processing On Mental Time T...,Istituti Clinici Scientifici Maugeri SpA,Istituti Clinici Scientifici Maugeri Spa,OTHER,OTHER,other,INTERVENTIONAL,66,1,True,ESTIMATED,Small,Mental Time Travel; Projection; Self/other dis...,"Brain Injuries, Focal",BEHAVIORAL; BEHAVIORAL; BEHAVIORAL,Mental Time Travel task,Mental Time Travel task; Age Estimation contro...,Healthy participants; not-vmPFC patients; vmPF...,"A Psychophysical Task (D'Angelo Et Al., 2023) ...",False,None,None,None,2025-02-12,2025-02-12,2024-12-01,2025-12-30,2025-06-30,Castel Goffredo,Italy,"lat: 45.29403, lon: 10.473",Istituti Clinici Scientifici Maugeri,46042,Mantova,ADULT; OLDER_ADULT,ALL,40,80,RECRUITING,active,2025-01-01,Likely Responses of events (Percentage),At baseline,"If the vmPFC plays a crucial role in MTT, by a...",Temporal estimation of cultural prototypical l...,At baseline,The deficit found in MTT may be related to an ...,False,"Brain Injuries, Focal",Mental Time Travel (Mtt) Refers To The Ability...,Mental Time Travel (Mtt) Is The Ability To Pro...,NONE,NON_RANDOMIZED,PARALLEL,BASIC_SCIENCE,False,394,394.000,BACKGROUND; BACKGROUND; BACKGROUND; BACKGROUND...,"Hutchinson JB, Uncapher MR, Wagner AD. Posteri...",19470649; 27313902; 15096584; 25579447; 216808...,Centro Studi e Ricerche in Neuroscienze Cognit...,UNKNOWN,NO,0.490,0.170,73.000,0.351,OTHER
PRIVATE,2025-02-26,private,"Baili-Bio (Chengdu) Pharmaceutical Co., Ltd.","Baili-Bio (Chengdu) Pharmaceutical Co., Ltd.",NCT06857214,A Randomized Controlled Phase I Clinical Study...,GNC-038-106,A Study Of Gnc-038 Tetra-Specific Antibody Inj...,"Sichuan Baili Pharmaceutical Co., Ltd.","Sichuan Baili Pharmaceutical Co., Ltd.",INDUSTRY,INDUSTRY,phase_1,INTERVENTIONAL,54,1,False,ESTIMATED,Small,nan,Systemic Lupus Erythematosus,DRUG; DRUG,GNC-038,GNC-038; Placebo,GNC-038; Placebo,Administration By Intravenous Infusion. Once A...,False,None,None,None,2025-03-04,2025-03-04,2025-03-01,2027-12-01,2027-12-01,Shanghai,China,"lat: 31.22222, lon: 121.45806","Renji Hospital, Shanghai Jiao Tong University ...",None,Shanghai,ADULT; OLDER_ADULT,ALL,18,75,NOT_YET_RECRUITING,active,2025-02-01,Phase Ia: Dose limiting toxicity (DLT); Phase ...,Up to approximately 28 days; Up to approximate...,DLTs are assessed according to NCI-CTCAE v5.0 ...,Anti-drug

In [42]:
df_apps["links"] = "https://clinicaltrials.gov/study/" + df_apps["trial_id"]

In [43]:
df_apps.head()

source             subsidiary                sponsor  \
ticker date                                                               
LLY    2022-01-03  listed  ELI LILLY AND COMPANY  Eli Lilly and Company   
       2022-01-03  listed  ELI LILLY AND COMPANY  Eli Lilly and Company   
       2022-01-03  listed  ELI LILLY AND COMPANY  Eli Lilly and Company   
       2022-01-05  listed  ELI LILLY AND COMPANY  Eli Lilly and Company   
       2022-01-05  listed      Loxo Oncology Inc    Loxo Oncology, Inc.   

                      trial_id  \
ticker date                      
LLY    2022-01-03  NCT05177094   
       2022-01-03  NCT05176314   
       2022-01-03  NCT05176314   
       2022-01-05  NCT05205447   
       2022-01-05  NCT05205447   

                                                      official_title  \
ticker date                                                            
LLY    2022-01-03  Randomized, Placebo-Controlled, Phase 2 Clinic...   
       2022-01-03  A Phase 1, Open-Label, Drug Interaction Study ...   
       2022-01-03  A Phase 1, Open-Label, Drug Interaction Study ...   
       2022-01-05  A Phase 1, Two-Part, Open-Label, Fixed-Sequenc...   
       2022-01-05  A Phase 1, Two-Part, Open-Label, Fixed-Sequenc...   

                   success_prediction  economic_effect  duration_prediction  \
ticker date                                                                   
LLY    2022-01-03               0.950            0.890              260.000   
       2022-01-03               0.920            0.830               99.000   
       2022-01-03               0.920            0.830               99.000   
       2022-01-05               0.230            0.520               97.000   
       2022-01-05               0.230            0.520               97.000   

                   success_composite  \
ticker date                            
LLY    2022-01-03              0.910   
       2022-01-03              0.848   
       2022-01-03              0.848   
       2022-01-05              0.339   
       2022-01-05              0.339   

                                                          links  
ticker date                                                      
LLY    2022-01-03  https://clinicaltrials.gov/study/NCT05177094  
       2022-01-03  https://clinicaltrials.gov/study/NCT05176314  
       2022-01-03  https://clinicaltrials.gov/study/NCT05176314  
       2022-01-05  https://clinicaltrials.gov/study/NCT05205447  
       2022-01-05  https://clinicaltrials.gov/study/NCT05205447

Trial Information

In [54]:
df_lly = sov.data("clinical/trials", tickers=["LLY"]) 

In [55]:
df_lly

source             subsidiary                sponsor  \
ticker date                                                               
LLY    1999-11-01  listed  ELI LILLY AND COMPANY  Eli Lilly and Company   
       1999-11-01  listed  ELI LILLY AND COMPANY  Eli Lilly and Company   
       1999-11-01  listed  ELI LILLY AND COMPANY  Eli Lilly and Company   
       2000-03-07  listed  ELI LILLY AND COMPANY  Eli Lilly and Company   
       2000-11-06  listed  ELI LILLY AND COMPANY  Eli Lilly and Company   
...                   ...                    ...                    ...   
       2025-04-18  listed  ELI LILLY AND COMPANY  Eli Lilly and Company   
       2025-04-21  listed  ELI LILLY AND COMPANY  Eli Lilly and Company   
       2025-04-21  listed  ELI LILLY AND COMPANY  Eli Lilly and Company   
       2025-04-21  listed  ELI LILLY AND COMPANY  Eli Lilly and Company   
       2025-04-21  listed  ELI LILLY AND COMPANY  Eli Lilly and Company   

                      trial_id  \
ticker date                      
LLY    1999-11-01  NCT00003670   
       1999-11-01  NCT00003669   
       1999-11-01  NCT00003906   
       2000-03-07  NCT00004865   
       2000-11-06  NCT00006459   
...                        ...   
       2025-04-18  NCT06945419   
       2025-04-21  NCT06948786   
       2025-04-21  NCT06948435   
       2025-04-21  NCT06952530   
       2025-04-21  NCT06948422   

                                                      official_title  \
ticker date                                                            
LLY    1999-11-01  Phase Ii Study Of Ly353381-Hc1 Administered To...   
       1999-11-01  Phase Ii Nonrandomized Study Of Ly353381-Hc1 I...   
       1999-11-01  Study Of Tamoxifen And Raloxifene (Star) For T...   
       2000-03-07  Phase Ii Study Of Cetuximab Anti-Epidermal Gro...   
       2000-11-06  Phase Iii Study Of Gemcitabine Plus Paclitaxel...   
...                                                              ...   
       2025-04-18  A Single-Dose And Multiple-Ascending Dose Stud...   
       2025-04-21  Promote-Fl: Pirtobrutinib And Mosunetuzumab To...   
       2025-04-21  A Master Protocol To Investigate The Efficacy ...   
       2025-04-21  A Master Protocol To Investigate The Efficacy ...   
       2025-04-21  A Master Protocol To Investigate The Efficacy ...   

                  sponsor_study_id  \
ticker date                          
LLY    1999-11-01             2145   
       1999-11-01             2141   
       1999-11-01        NSABP P-2   
       2000-03-07    CDR0000067521   
       2000-11-06             2017   
...                            ...   
       2025-04-18            27267   
       2025-04-21        RG1125134   
       2025-04-21            27359   
       2025-04-21            27360   
       2025-04-21            27223   

                                                         brief_title  \
ticker date                                                            
LLY    1999-11-01  Hormone Therapy With Arzoxifene Hydrochloride ...   
       1999-11-01  Hormone Therapy With Arzoxifene Hydrochloride ...   
       1999-11-01  Study Of Tamoxifen And Raloxifene (Star) For T...   
       2000-03-07  Cetuximab Plus Cisplatin In Treating Patients ...   
       2000-11-06  Paclitaxel With Or Without Gemcitabine In Trea...   
...                                                              ...   
       2025-04-18  A Study Of Ly4086940 In Healthy Participants A...   
       2025-04-21  Pirtobrutinib And Mosunetuzumab For The Treatm...   
       2025-04-21  A Master Protocol Study Of Orforglipron (Ly350...   
       2025-04-21  A Master Protocol Study Of Orforglipron (Ly350...   
       2025-04-21  A Master Protocol For Orforglipron (Ly3502970)...   

                                      lead_sponsor  \
ticker date                                          
LLY    1999-11-01            Eli Lilly and Company   
       1999-11-01            Eli Lilly and Company   
       1999-11-